In [1]:
import os
import sys
import operator
import numpy as np
import pandas as pd
from scipy import sparse
import xgboost as xgb
from sklearn import model_selection, preprocessing, ensemble
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.metrics.scorer import make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from XGBoostPackage import xgbClass
from CrossValidation2 import CVScore
from Stacking import Ensemble, EnsembleClassifier
from Stacking2 import Ensemble2, EnsembleClassifier21, EnsembleClassifier22
%matplotlib inline

/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
X_train=pd.read_csv('X_train5_200feat.csv').iloc[:,1:]
X_test=pd.read_csv('X_test5_200feat.csv').iloc[:,1:]
y_train=pd.read_csv('y_train.csv')['interest_level'].ravel()#low0, medium1, high2

In [3]:
# index=[True]*len(X_train)
# for i in [0,1,2]:
#     index=index&~pd.isnull(X_train.iloc[:,i])
# X_train=X_train[index]
# y_train=y_train[index]

In [4]:
X_train.shape, y_train.shape, X_test.shape

((49352, 245), (49352,), (74659, 245))

In [5]:
n_folds=3
base_models=[RandomForestClassifier(n_estimators=500, max_features='auto', max_depth=25),\
            xgbClass(objective='multi:softprob', eva_metric="mlogloss", colsample_bytree=.7, eta=.02, \
                      max_depth=6, min_child_weight=1, num_class=3, subsample=.7, silent=1, seed=321,
                     num_rounds=2000), 
            ExtraTreesClassifier(n_estimators=500, max_features='auto', max_depth=25)]
ensembleObj=EnsembleClassifier22(n_folds=n_folds, n_class=3, 
                                 stacker=xgbClass(objective='multi:softprob', \
                                                                         eva_metric="mlogloss", \
                                           colsample_bytree=.7, eta=.1, max_depth=6, min_child_weight=1, \
                                           num_class=3, subsample=.7, silent=1), \
                     base_models=base_models, random_state=17)#,\

In [6]:
pred=ensembleObj.fit_predict_proba(T=X_test, X=X_train, y=y_train)
#pred=ensembleObj.fit_predict(T=X_test, X=X_train, y=y_train)
del ensembleObj

startig training model 0, training set 0
startig predicting model 0, training set 0
startig training model 0, training set 1
startig predicting model 0, training set 1
startig training model 0, training set 2
startig predicting model 0, training set 2
startig training model 1, training set 0
startig predicting model 1, training set 0
startig training model 1, training set 1
startig predicting model 1, training set 1
startig training model 1, training set 2
startig predicting model 1, training set 2
startig training model 2, training set 0
startig predicting model 2, training set 0
startig training model 2, training set 1
startig predicting model 2, training set 1
startig training model 2, training set 2
startig predicting model 2, training set 2


In [ ]:
###################
###################

In [6]:
X_fit, X_val, y_fit, y_val=train_test_split(X_train[:5000], y_train[:5000], test_size=0.33, random_state=42)

In [7]:
n_folds=5
base_models=[RandomForestClassifier(n_estimators=1000, max_features='auto', max_depth=25),\
            xgbClass(objective='multi:softprob', eva_metric="mlogloss", colsample_bytree=.7, eta=.02, \
                      max_depth=6, min_child_weight=1, num_class=3, subsample=.7, silent=1, seed=321,
                     num_rounds=2000), 
            ExtraTreesClassifier(n_estimators=500, max_features='auto', max_depth=25)]
ensembleObj=EnsembleClassifier2(n_folds=n_folds, n_class=3, stacker=xgbClass(objective='multi:softprob', \
                                                                         eva_metric="mlogloss", \
                                           colsample_bytree=.7, eta=.1, max_depth=6, min_child_weight=1, \
                                           num_class=3, subsample=.7, silent=1), \
                     base_models=base_models, random_state=17)#,\

In [8]:
y_pred=ensembleObj.fit_predict_proba(T=X_val, X=X_fit, y=y_fit)
log_loss(y_val, y_pred)

startig training model 0, training set 0
startig predicting model 0, training set 0
startig training model 0, training set 1
startig predicting model 0, training set 1
startig training model 0, training set 2
startig predicting model 0, training set 2
startig training model 0, training set 3
startig predicting model 0, training set 3
startig training model 0, training set 4
startig predicting model 0, training set 4
startig training model 1, training set 0
startig predicting model 1, training set 0
startig training model 1, training set 1
startig predicting model 1, training set 1
startig training model 1, training set 2
startig predicting model 1, training set 2
startig training model 1, training set 3
startig predicting model 1, training set 3
startig training model 1, training set 4
startig predicting model 1, training set 4
startig training model 2, training set 0
startig predicting model 2, training set 0
startig training model 2, training set 1
startig predicting model 2, trainin

0.75284108823333307

In [9]:
model=xgbClass(objective='multi:softprob', eva_metric="mlogloss", colsample_bytree=.7, eta=.02, \
                      max_depth=6, min_child_weight=1, num_class=3, subsample=.7, silent=1, seed=321,
                     num_rounds=2000)
model.fit(X_fit, y_fit)
y_pred=model.predict_proba(X_val)
log_loss(y_val, y_pred)

0.74512806818289856

In [ ]:
#################
#################

In [6]:
n_folds=5
base_models=[[RandomForestClassifier(n_estimators=1000, max_features='auto', max_depth=25) for i in np.arange(n_folds)],\
            [xgbClass(objective='multi:softprob', eva_metric="mlogloss", colsample_bytree=.7, eta=.02, \
                      max_depth=6, min_child_weight=1, num_class=3, subsample=.7, silent=1, seed=321,
                     num_rounds=2000) \
             for i in np.arange(n_folds)], 
            [ExtraTreesClassifier(n_estimators=500, max_features='auto', max_depth=25) for i in np.arange(n_folds)]]
ensembleObj=EnsembleClassifier(n_folds=n_folds, n_class=3, stacker=xgbClass(objective='multi:softprob', \
                                                                         eva_metric="mlogloss", \
                                           colsample_bytree=.7, eta=.1, max_depth=6, min_child_weight=1, \
                                           num_class=3, subsample=.7, silent=1), \
                     base_models=base_models, is_TimeSeries=False, random_state=17)#,\
ensembleObj.fit(X_train=X_train, y_train=y_train)

startig training model 0, training set 0
startig training model 0, training set 1
startig training model 0, training set 2
startig training model 0, training set 3
startig training model 0, training set 4
startig training model 1, training set 0


KeyboardInterrupt: 

In [ ]:
pred=ensembleObj.predict_proba(X_test=X_test)

In [ ]:
n_folds=5
base_models=[[RandomForestClassifier(n_estimators=1000, max_features='auto', max_depth=25) for i in np.arange(n_folds)],\
            [xgbClass(objective='multi:softprob', eva_metric="mlogloss", colsample_bytree=.7, eta=.02, \
                      max_depth=6, min_child_weight=1, num_class=3, subsample=.7, silent=1, seed=321,
                     num_rounds=2000) \
             for i in np.arange(n_folds)], 
            [ExtraTreesClassifier(n_estimators=500, max_features='auto', max_depth=25) for i in np.arange(n_folds)]]
ensembleObj=EnsembleClassifier(n_folds=n_folds, n_class=3, stacker=xgbClass(objective='multi:softprob', \
                                                                         eva_metric="mlogloss", \
                                           colsample_bytree=.7, eta=.1, max_depth=6, min_child_weight=1, \
                                           num_class=3, subsample=.7, silent=1), \
                     base_models=base_models, is_TimeSeries=False, random_state=17)#,\
ensembleObj.fit(X_train=X_train, y_train=y_train)

In [7]:
out_df = pd.DataFrame(pred)
out_df.columns = ["low", "medium", "high"]
out_df["listing_id"] = pd.read_json("../input/test.json").listing_id.values
out_df.to_csv("apr25_1_Stack.csv", index=False)